In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("LungCancerEDA").getOrCreate()

df = spark.read.csv("/content/Lung Cancer.csv", header=True, inferSchema=True)

# Standardize column names
df = df.toDF(*[c.lower().replace(" ", "_") for c in df.columns])


## 1. How many rows and columns are in the dataset?

In [ ]:
print(f"Rows: {df.count()}, Columns: {len(df.columns)}")


Rows: 890000, Columns: 17


## 2. Gender distribution of patients

In [ ]:
df.groupBy("gender").count().orderBy("count", ascending=False).show()


+------+------+
|gender| count|
+------+------+
|  Male|445134|
|Female|444866|
+------+------+



## 3. Average and median age of patients

In [ ]:
df.select(
    F.round(F.avg("age"),2).alias("avg_age"),
    F.expr("percentile(age, 0.5)").alias("median_age")
).show()


+-------+----------+
|avg_age|median_age|
+-------+----------+
|  55.01|      55.0|
+-------+----------+



## 4. Cancer stage distribution

In [ ]:
df.groupBy("cancer_stage").count().orderBy("cancer_stage").show()


+------------+------+
|cancer_stage| count|
+------------+------+
|     Stage I|222516|
|    Stage II|222363|
|   Stage III|222594|
|    Stage IV|222527|
+------------+------+



## 5. Family history vs Lung Cancer survival.

In [ ]:
df.groupBy("family_history","survived").count().orderBy("family_history").show()


+--------------+--------+------+
|family_history|survived| count|
+--------------+--------+------+
|            No|       1| 97798|
|            No|       0|347383|
|           Yes|       0|346613|
|           Yes|       1| 98206|
+--------------+--------+------+



## 6. Smoking status vs Cancer stage

In [ ]:
df.groupBy("smoking_status","cancer_stage").count().orderBy("smoking_status").show()


+--------------+------------+-----+
|smoking_status|cancer_stage|count|
+--------------+------------+-----+
|Current Smoker|    Stage IV|55181|
|Current Smoker|     Stage I|55578|
|Current Smoker|   Stage III|55710|
|Current Smoker|    Stage II|55429|
| Former Smoker|    Stage IV|55556|
| Former Smoker|   Stage III|55440|
| Former Smoker|    Stage II|55699|
| Former Smoker|     Stage I|55486|
|  Never Smoked|    Stage IV|55942|
|  Never Smoked|    Stage II|55466|
|  Never Smoked|   Stage III|55578|
|  Never Smoked|     Stage I|55765|
|Passive Smoker|     Stage I|55687|
|Passive Smoker|    Stage IV|55848|
|Passive Smoker|    Stage II|55769|
|Passive Smoker|   Stage III|55866|
+--------------+------------+-----+



## 7. Average BMI & Cholesterol by survival outcome

In [ ]:
df.groupBy("survived").agg(
    F.round(F.avg("bmi"),2).alias("avg_bmi"),
    F.round(F.avg("cholesterol_level"),2).alias("avg_cholesterol")
).show()


+--------+-------+---------------+
|survived|avg_bmi|avg_cholesterol|
+--------+-------+---------------+
|       1|  30.49|         233.65|
|       0|  30.49|         233.63|
+--------+-------+---------------+



## 8. Comorbidities analysis (Hypertension, Asthma, Cirrhosis, Other Cancer) prevalence

In [ ]:
comorbidities = ["hypertension","asthma","cirrhosis","other_cancer"]

comorbidity_counts = [(c, df.filter(F.col(c)=="YES").count()) for c in comorbidities]
spark.createDataFrame(comorbidity_counts, ["condition","count"]).show()


+------------+-----+
|   condition|count|
+------------+-----+
|hypertension|    0|
|      asthma|    0|
|   cirrhosis|    0|
|other_cancer|    0|
+------------+-----+



## 9. Treatment types vs Survival Rate

In [ ]:
df.groupBy("treatment_type","survived").count().orderBy("treatment_type").show()


+--------------+--------+------+
|treatment_type|survived| count|
+--------------+--------+------+
|  Chemotherapy|       1| 48836|
|  Chemotherapy|       0|174426|
|      Combined|       0|173607|
|      Combined|       1| 49002|
|     Radiation|       1| 48714|
|     Radiation|       0|172154|
|       Surgery|       1| 49452|
|       Surgery|       0|173809|
+--------------+--------+------+



## 10. Survival trend over time (by diagnosis year)

In [ ]:
df = df.withColumn("year", F.year("diagnosis_date"))
df.groupBy("year","survived").count().orderBy("year").show()


+----+--------+-----+
|year|survived|count|
+----+--------+-----+
|2014|       0|40382|
|2014|       1|11387|
|2015|       0|69343|
|2015|       1|19657|
|2016|       0|69558|
|2016|       1|19694|
|2017|       0|69317|
|2017|       1|19531|
|2018|       1|19391|
|2018|       0|69095|
|2019|       1|19488|
|2019|       0|69990|
|2020|       1|19761|
|2020|       0|69424|
|2021|       0|69309|
|2021|       1|19730|
|2022|       0|69370|
|2022|       1|19561|
|2023|       0|69133|
|2023|       1|19714|
+----+--------+-----+
only showing top 20 rows

